In [ ]:
# Paso 2: Importar librerías
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
import torch
from transformers import EarlyStoppingCallback

# Paso 3: Cargar y preparar el dataset
df = pd.read_csv("../data/mails_dataset.csv")  

# Concatenar asunto y cuerpo separados por espacio
df['text_combined'] = df['subject'].fillna('') + ' </s> ' + df['text'].fillna('')

# Nos centramos en la columna combinada y la etiqueta priority
df = df[['text_combined', 'priority']].dropna()

# Codificar etiquetas
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["priority"])

# Guardar las etiquetas para decodificar luego
label2id = {label: i for i, label in enumerate(label_encoder.classes_)}
id2label = {i: label for label, i in label2id.items()}

# Paso 4: Crear Dataset de Hugging Face
dataset = Dataset.from_pandas(df.rename(columns={"text_combined": "text", "label": "label"}))

# Paso 5: Tokenización
model_name = "pysentimiento/robertuito-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

dataset = dataset.map(tokenize, batched=True)

# División entrenamiento y validación
dataset = dataset.train_test_split(test_size=0.2)

# Paso 6: Cargar modelo preentrenado
num_labels = len(label2id)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

# Paso 7: Configurar entrenamiento
training_args = TrainingArguments(
    output_dir="./priority_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,      
    metric_for_best_model="f1",        
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=6,
    weight_decay=0.01,
    logging_dir="./logs_priority",
    logging_steps=10
)

# Paso 8: Función de evaluación
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    return {"accuracy": acc, "f1": f1}

# Paso 9: Entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

# Paso 10: Entrenar
trainer.train()

# Paso 11: Guardar el modelo y tokenizer
trainer.save_model("./priority_model")
tokenizer.save_pretrained("./priority_model")


c:\Users\Usuario\Desktop\MailClassifier\.venv\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/222 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pysentimiento/robertuito-base-uncased and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/138 [00:00<?, ?it/s]

c:\Users\Usuario\Desktop\MailClassifier\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


...

c:\Users\Usuario\Desktop\MailClassifier\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'loss': 0.2414, 'grad_norm': 2.9588441848754883, 'learning_rate': 2.6086956521739132e-06, 'epoch': 5.22}
{'loss': 0.2669, 'grad_norm': 5.157289028167725, 'learning_rate': 1.1594202898550726e-06, 'epoch': 5.65}


  0%|          | 0/6 [00:00<?, ?it/s]

{'eval_loss': 0.39798328280448914, 'eval_accuracy': 0.8222222222222222, 'eval_f1': 0.819965811965812, 'eval_runtime': 1.0766, 'eval_samples_per_second': 41.8, 'eval_steps_per_second': 5.573, 'epoch': 6.0}
{'train_runtime': 135.032, 'train_samples_per_second': 7.865, 'train_steps_per_second': 1.022, 'train_loss': 0.4974019587903783, 'epoch': 6.0}


('./priority_model\\tokenizer_config.json',
 './priority_model\\special_tokens_map.json',
 './priority_model\\tokenizer.json')

In [16]:
from sklearn.metrics import classification_report
import numpy as np

# Obtener predicciones en el conjunto de validación
predictions = trainer.predict(dataset["test"])
y_true = predictions.label_ids
y_pred = np.argmax(predictions.predictions, axis=1)

# Mostrar el classification report con nombres reales
print("Reporte de clasificación por clase:")
print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))


c:\Users\Usuario\Desktop\MailClassifier\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


  0%|          | 0/6 [00:00<?, ?it/s]

Reporte de clasificación por clase:
              precision    recall  f1-score   support

        alta       0.89      0.94      0.92        18
        baja       0.92      0.79      0.85        14
       media       0.71      0.77      0.74        13

    accuracy                           0.84        45
   macro avg       0.84      0.83      0.84        45
weighted avg       0.85      0.84      0.84        45



In [5]:
from transformers import pipeline

# Cargar el modelo y tokenizer ya entrenados para PRIORITY
clf_priority = pipeline("text-classification", model="./modelo_priority", tokenizer="./modelo_priority")

# Lista de ejemplos: (texto, prioridad_esperada)
ejemplos_priority = [
    ("Mi pedido no ha llegado y ya pasaron 10 días", "alta"),
    ("El repartidor fue descortés", "media"),
    ("¿Pueden confirmarme si tienen stock del producto?", "media"),
    ("Gracias por la atención, todo perfecto", "baja"),
    ("Recibí una caja vacía sin el producto dentro", "alta"),
    ("Quisiera modificar la dirección de entrega", "media"),
    ("Nos gustaría evaluar una posible alianza comercial", "baja"),
    ("El servicio fue correcto, pero la web es un poco confusa", "media"),
    ("Solicito el alta inmediata del servicio para un cliente", "alta"),
    ("No me llegó el correo de activación, pero puedo esperar", "baja"),
]

# Imprimir resultados
print(f"{'Texto':<60} | {'Esperado':<6} | {'Predicción':<10} | {'Score'}")
print("-" * 100)

for texto, esperado in ejemplos_priority:
    pred = clf_priority(texto)[0]
    print(f"{texto[:57]:<60} | {esperado:<8} | {pred['label']:<10} | {pred['score']:.4f}")


Texto                                                        | Esperado | Predicción | Score
----------------------------------------------------------------------------------------------------
Mi pedido no ha llegado y ya pasaron 10 días                 | alta     | alta       | 0.4748
El repartidor fue descortés                                  | media    | media      | 0.4731
¿Pueden confirmarme si tienen stock del producto?            | media    | media      | 0.8573
Gracias por la atención, todo perfecto                       | baja     | baja       | 0.7073
Recibí una caja vacía sin el producto dentro                 | alta     | alta       | 0.5957
Quisiera modificar la dirección de entrega                   | media    | media      | 0.7559
Nos gustaría evaluar una posible alianza comercial           | baja     | media      | 0.5662
El servicio fue correcto, pero la web es un poco confusa     | media    | media      | 0.9214
Solicito el alta inmediata del servicio para un client